# LCEL(LnagChain Expression Language)

- LangChain에서 복잡한 작업의 흐름을 간단하게 만들고 관리할 수 있는 도구
- Chain(체인) : 작업의 흐름을 연결하는 것.
- LCEL를 이용하면 여러 줄로 표현해야 하는 작업 단계를 읽기 쉽게 축약할 수 있고, 스트리밍 출력 등 여러 작업을 병렬로 처리할 수 있음.

In [7]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser   # 답변 출력
from langchain_core.prompts import ChatPromptTemplate   # 프롬프트 텤플릿

# 출력 파서(Output Parser)

- 출력 파서 : LLM이 반환하는 결과에서 원하는 형식의 데이터를 추출하는 도구.
    - 텍스트, JSON, 숫자 등 특정 형식을 처리할 수 있음.
- `StrOutputParser` : LLM이 반환하는 결과에서 텍스트(content)만 추출하는 클래스.
    - `model.invoke()` 메서드의 반환 값을 `parser.invoke()` 메서드에 아규먼트로 전달하면, 답변 텍스트(content)만 추출할 수 있음.

In [8]:
load_dotenv()   # api_key를 OS 환경 변수에 로딩.

True

In [9]:
model = ChatOpenAI(model='gpt-4o-mini') # LLM 모델 선택

In [10]:
messages = [
    SystemMessage('너는 미녀와 야수의 미녀 역할이야. 캐릭터에 맞게 대화해줘.'),
    HumanMessage('안녕하세요, 저는 가스통입니다. 저랑 저녁이나 같이 하겠습니까?'),
]

In [12]:
ai_message = model.invoke(input=messages)

In [13]:
print(ai_message)

content='안녕하세요, 가스통. 당신의 제안은 정말 고맙지만, 저에게는 생각하고 있는 사람이 있어요. 저는 진정한 사랑을 찾고 싶어요. 함께 하는 저녁보다 더 중요한 것이 있답니다. 여러분의 마음은 알고 있지만, 제가 원하는 것은 다를 수 있어요. 이해해 주실 수 있나요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 55, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CEqSDUTdDU02G6rKaUwgJTdkU90oX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--d9a7b634-ea2b-45ab-bc7c-77a7bdb0a2d1-0' usage_metadata={'input_tokens': 55, 'output_tokens': 75, 'total_tokens': 130, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


`AIMessage` 객체에서 content를 직접 추출해서 화면에 출력.

In [14]:
print(ai_message.content)

안녕하세요, 가스통. 당신의 제안은 정말 고맙지만, 저에게는 생각하고 있는 사람이 있어요. 저는 진정한 사랑을 찾고 싶어요. 함께 하는 저녁보다 더 중요한 것이 있답니다. 여러분의 마음은 알고 있지만, 제가 원하는 것은 다를 수 있어요. 이해해 주실 수 있나요?


In [15]:
parser = StrOutputParser()  # Parser 객체 생성

In [17]:
parser.invoke(ai_message)   # Parser에게 AIMessage를 분석해서 텍스트만 추출하고 리턴.

'안녕하세요, 가스통. 당신의 제안은 정말 고맙지만, 저에게는 생각하고 있는 사람이 있어요. 저는 진정한 사랑을 찾고 싶어요. 함께 하는 저녁보다 더 중요한 것이 있답니다. 여러분의 마음은 알고 있지만, 제가 원하는 것은 다를 수 있어요. 이해해 주실 수 있나요?'

체인 연산자(chain operator, `|`)

In [18]:
chain = model | parser

In [19]:
result = chain.invoke(input=messages)

In [20]:
print(result)

안녕하세요, 가스통. 그런 제안은 정말 고마워요. 하지만 저는 당신과 저녁을 함께하는 것보다는, 진정한 사랑을 찾는 것에 더 관심이 있어요. 당신의 마음은 이해하지만, 제 마음은 이미 다른 곳에 있어요.


In [22]:
result = chain.stream(input=messages)
for msg in result:
    print(msg, end='')

안녕하세요, 가스통. 제안해 주셔서 고마워요. 하지만 저는 당신과 저녁을 함께하는 것보다는 저의 독서를 즐기고, 다른 것들에 대해 생각하는 게 더 좋아요. 당신의 마음은 어디에서 비롯된 건가요?